This notebook continuously estimates the prevailing, realtime, positivity values via 

* $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$

curves, wherein

$\qquad \qquad \qquad positives_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{P_{\delta}}  \right) \times \frac{1}{population}$


$\qquad \qquad \qquad \qquad  tests_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{T_{\delta}} \right) \times \frac{1}{population}$  

<br>

which are the positive cases per hundred thousand and test numbers per hundred thousand, respectively, based on their cumulative values and

<br>

<table style="width:35%; text-align: left; border: 0px solid black; float:left; margin-left: 50px">
    <tr>
        <th style="width:20%;">$\mathcal{Variable} \qquad$</th><th></th> 
    </tr>
    <tr>
        <td>$\tau$</td><td>end date</td>
    </tr>
    <tr>
        <td>$\delta$</td><td>a series date</td>
    </tr>
    <tr>
      <td>$P_{\delta}$</td><td>The number of positive cases on date $\delta$.</td>
    </tr>
    <tr>
      <td>$T_{\delta}$</td><td>The number of tests on date $\delta$.</td>
    </tr>
    
</table>


<br>

<br>

The subscript $c$ denotes cumulative.   The graph image below illustrates the curves in question.  In brief 

* The quotient of the coordinates of a point along a curve is the cumulative positivity value. 

* The gradient of a modelled secant line to a sequential set of curve rear points estimates the prevailing positivity value.

* The aforementioned secant line model predicts the expected values of $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}}$ increases.


<div style="margin-left: 100px;">
    <img src="https://github.com/briefings/sars/raw/develop/fundamentals/atlantic/docs/ptr.png" longdesc="surveillance" width="40%" style="float:center">
</div>

<br>

Upcoming: Link this image to the interactive graph.



<br>

## Preliminaries

### Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask

import logging

import os
import pathlib
import sys

import statsmodels.regression.linear_model


In [2]:
from statsmodels.formula.api import ols

<br>

### Paths

In [3]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [4]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'prospects')

<br>

Appending paths

In [5]:
sys.path.append(parent)

<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>

### Custom

Import

In [7]:
import cartographs.boundaries.us.boundaries
import cartographs.boundaries.us.settings

In [8]:
import algorithms.curves.estimates
import algorithms.curves.predictions
import algorithms.curves.secants

In [9]:
import atlantic.base.directories
import atlantic.investigations.prospects
import atlantic.investigations.hospitalizations

<br>

Set-up directories

In [10]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

### Classes

**Boundaries**

In [11]:
settings = cartographs.boundaries.us.settings.Settings()
boundaries = cartographs.boundaries.us.boundaries.Boundaries(settings.crs)

In [12]:
states = boundaries.states(year=settings.latest)
states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


<br>
<br>

## Data

<br>

**Attributes**

In [13]:
fields = ['datetimeobject', 'STUSPS', 'positiveIncrease', 'testIncrease', 'deathIncrease', 'hospitalizedIncrease', 
          'positiveCumulative', 'testCumulative',  'deathCumulative', 'hospitalizedCumulative', 
          'positiveRate', 'testRate', 'deathRate', 'hospitalizedRate', 'ndays']

dtype = {'STUSPS': 'str', 'positiveIncrease': np.float64, 'testIncrease': np.float64, 'deathIncrease': np.float64, 
         'hospitalizedIncrease': np.float64, 'positiveCumulative': np.float64, 'testCumulative': np.float64, 
         'deathCumulative': np.float64, 'hospitalizedCumulative': np.float64, 'positiveRate': np.float64, 
         'testRate': np.float64, 'deathRate': np.float64, 'hospitalizedRate': np.float64, 'ndays': np.int64}        

parse_dates = ['datetimeobject']

<br>
<br>

**The URL strings of the sources**

In [14]:
uribaseline = os.path.join(parent, 'warehouse', 'baselines.csv')
urihospitalizations = os.path.join(parent, 'warehouse', 'hospitalizations.csv')

<br>
<br>

**Baseline**

In [15]:
baselines = pd.read_csv(filepath_or_buffer=uribaseline, header=0, usecols=fields, 
                        dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

baselines.loc[:, 'positiveTestRate'] = np.where(
    baselines['testRate'] > 0, 100 * baselines['positiveRate'] / baselines['testRate'], 0)
baselines.loc[:, 'deathPositiveRate'] = np.where(
    baselines['positiveRate'] > 0, 100 * baselines['deathRate'] / baselines['positiveRate'], 0)

logger.info(baselines.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19812 entries, 0 to 19811
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetimeobject          19812 non-null  datetime64[ns]
 1   STUSPS                  19812 non-null  object        
 2   deathIncrease           19812 non-null  float64       
 3   deathCumulative         19812 non-null  float64       
 4   positiveIncrease        19812 non-null  float64       
 5   positiveCumulative      19812 non-null  float64       
 6   hospitalizedIncrease    19812 non-null  float64       
 7   hospitalizedCumulative  19812 non-null  float64       
 8   testIncrease            19812 non-null  float64       
 9   testCumulative          19812 non-null  float64       
 10  deathRate               19812 non-null  float64       
 11  positiveRate            19812 non-null  float64       
 12  testRate                19812 non-null  float6

<br>
<br>

**Hospitalizations**

In [16]:
hospitalizations = pd.read_csv(filepath_or_buffer=urihospitalizations, header=0, 
                               usecols=fields,dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

hospitalizations.loc[:, 'hospitalizedPositiveRate'] =  np.where(
    hospitalizations['positiveRate'] > 0, 100 * hospitalizations['hospitalizedRate'] / hospitalizations['positiveRate'], 0)
hospitalizations.loc[:, 'deathHospitalizedRate'] =  np.where(
    hospitalizations['hospitalizedRate'] > 0, 100 * hospitalizations['deathRate'] / hospitalizations['hospitalizedRate'], 0)

logger.info(hospitalizations.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14097 entries, 0 to 14096
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetimeobject            14097 non-null  datetime64[ns]
 1   STUSPS                    14097 non-null  object        
 2   deathIncrease             14097 non-null  float64       
 3   deathCumulative           14097 non-null  float64       
 4   positiveIncrease          14097 non-null  float64       
 5   positiveCumulative        14097 non-null  float64       
 6   hospitalizedIncrease      14097 non-null  float64       
 7   hospitalizedCumulative    14097 non-null  float64       
 8   testIncrease              14097 non-null  float64       
 9   testCumulative            14097 non-null  float64       
 10  deathRate                 14097 non-null  float64       
 11  positiveRate              14097 non-null  float64       
 12  testRate          

<br>
<br>

## Regression

<br>

**Latest Date**

In [17]:
latestdate = baselines.datetimeobject.max()
logger.info(type(latestdate))
logger.info(latestdate)

INFO:__main__:<class 'pandas._libs.tslibs.timestamps.Timestamp'>
INFO:__main__:2021-02-05 00:00:00


<br>
<br>

**Periods**

Within the context of the aforementioned $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$ curves, each value of this array is the number of data points, starting from the end of a curve, that are used to develop **a regression model of the tangent/secant line to the points**.

Key points:

* The model predicts the expected $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}, \tau}$ increases.

* The gradient of each model is an estimate of the prevailing positivity rate; an estimate of the current daily positivity value. [cf. continous positivity value] 

In [18]:
periods = np.arange(8, 29)
logger.info( periods)

INFO:__main__:[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]


<br>
<br>

**Partitions**

In [19]:
partitionby = 'STUSPS'
partitions = baselines[partitionby].unique()
logger.info(partitions)

INFO:__main__:['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'PR']


<br>
<br>

### **positives vs. tests**

<br>

**Analysis**

In [20]:
formula = 'positiveRate ~ testRate'
regressor = 'testRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=100, num=5, latestdate=latestdate)

In [21]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of affairs**

In [22]:
ptr = estimates

<br>

Prevailing Positive Test Rate

In [23]:
ptr.loc[:, 'prevailingPTR'] = 100 * ptr['gradient']
ptr = ptr.merge(baselines[['datetimeobject', 'STUSPS', 'positiveTestRate']], how='left', on=['datetimeobject', 'STUSPS'])
ptr.loc[:, 'cf'] = ptr[['prevailingPTR', 'positiveTestRate']].max(axis=1)

ptr.loc[:, 'rank'] = ptr['cf'].rank(method='min', ascending=False)
ptr.sort_values(by='rank', ascending=True, inplace=True)
ptr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [24]:
ptr.loc[:, 'prevailingTPC'] = np.where(ptr['prevailingPTR'] > 0, ptr['prevailingPTR'].rdiv(100), 0)
ptr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'positiveTestRate': 'continuousPTR'},
          inplace=True)
ptr.to_csv(path_or_buf=os.path.join(warehouse, 'ptr.csv'), index=False, header=True, encoding='utf-8')
ptr.to_json(path_or_buf=os.path.join(warehouse, 'ptr.json'), orient='values')

ptr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingPTR,continuousPTR,rank,prevailingTPC
13,2021-02-05,0.0406226,0.0325717,0.0499579,3.35115e-18,0.991987,3747.33,IL,4.06226,6.931291,48.0,24.6169
48,2021-02-05,0.0664322,0.0523859,0.0698774,2.38118e-20,0.996064,-464.06,WV,6.64322,6.197699,49.0,15.0529
22,2021-02-05,0.0473813,0.0414251,0.0552313,1.66743e-22,0.996186,1668.87,MI,4.73813,6.471163,50.0,21.1054
6,2021-02-05,0.0383969,0.0312587,0.0471071,1.22502e-15,0.990363,866.455,CT,3.83969,4.352265,51.0,26.0438
19,2021-02-05,0.0366159,0.0184841,0.0473394,4.70197e-14,0.972216,-866.558,ME,3.66159,2.837283,52.0,27.3105


<br>
<br>

### **deaths vs. positives**

In [25]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [26]:
formula = 'deathRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=20, num=5, latestdate=latestdate)

In [27]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of Affairs**

In [28]:
dpr = estimates

<br>

Prevailing Death Positive Rate

In [29]:
dpr.loc[:, 'prevailingDPR'] = 100 * dpr['gradient']
dpr = dpr.merge(baselines[['datetimeobject', 'STUSPS', 'deathPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
dpr.loc[:, 'cf'] = dpr[['prevailingDPR', 'deathPositiveRate']].max(axis=1)

dpr.loc[:, 'rank'] = dpr['cf'].rank(method='min', ascending=False)
dpr.sort_values(by='rank', ascending=True, inplace=True)
dpr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [30]:
dpr.loc[:, 'prevailingCPD'] = np.where(dpr['prevailingDPR'] > 0, dpr['prevailingDPR'].rdiv(100), 0)
dpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'deathPositiveRate': 'continuousDPR'},
          inplace=True)
dpr.to_csv(path_or_buf=os.path.join(warehouse, 'dpr.csv'), index=False, header=True, encoding='utf-8')
dpr.to_json(path_or_buf=os.path.join(warehouse, 'dpr.json'), orient='values')

dpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDPR,continuousDPR,rank,prevailingCPD
5,2021-02-05,0.0121104,0.00993234,0.0138485,1.3558e-20,0.995147,14.3603,CO,1.21104,1.416390,48.0,82.5735
1,2021-02-05,0.0140633,0.00719162,0.0387149,2.03011e-06,0.765342,-64.2659,AK,1.40633,0.523658,49.0,71.1069
46,2021-02-05,0.0122834,0.00966293,0.0176457,3.64598e-16,0.980224,3.14722,VA,1.22834,1.290973,50.0,81.4108
27,2021-02-05,0.0103708,0.00451868,0.0180018,3.95188e-13,0.965556,-2.72246,NE,1.03708,1.014145,51.0,96.4248
44,2021-02-05,0.00792367,0.00668613,0.0112008,5.24238e-17,0.988415,-33.5467,UT,0.792367,0.490228,52.0,126.204


<br>
<br>

### **hospitalizations vs. positives**

In [31]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [32]:
formula = 'hospitalizedRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [33]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [34]:
hpr = estimates

<br>

Prevailing Hospitalised Positive Rate

In [35]:
hpr.loc[:, 'prevailingHPR'] = 100 * hpr['gradient']
hpr.loc[:, 'rank'] = hpr['prevailingHPR'].rank(method='min', ascending=False)
hpr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [36]:
hpr = hpr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'hospitalizedPositiveRate']], 
                how='left', on=['datetimeobject', 'STUSPS'])
hpr.loc[:, 'prevailingCPH'] = np.where(hpr['prevailingHPR'] > 0, hpr['prevailingHPR'].rdiv(100), 0)

hpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'hospitalizedPositiveRate': 'continuousHPR'}, 
           inplace=True)

hpr.to_csv(path_or_buf=os.path.join(warehouse, 'hpr.csv'), index=False, header=True, encoding='utf-8')
hpr.to_json(path_or_buf=os.path.join(warehouse, 'hpr.json'), orient='values')

hpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingHPR,rank,continuousHPR,prevailingCPH
32,2021-02-05,0.017965,0.0047387,0.0428644,5.02651e-08,0.851349,125.925,MA,1.7965,33.0,3.425489,55.6637
33,2021-02-05,0.0109749,0.00482717,0.0405529,2.95029e-07,0.815474,191.452,MS,1.09749,34.0,3.137534,91.1172
34,2021-02-05,0.00865482,0.00547465,0.0101361,8.01937e-19,0.986052,34.1412,NH,0.865482,35.0,1.553369,115.543
35,2021-02-05,2.77556e-17,-1.84634e-14,1.89075e-14,0.921727,-217.895,343.787,CT,2.77556e-15,36.0,4.725645,3.60288e+16
36,2021-02-05,2.77556e-17,-1.73357e-14,1.889e-14,0.774652,-156.889,462.615,NY,2.77556e-15,36.0,6.208714,3.60288e+16


<br>
<br>

### **deaths vs. hospitalizations**

In [37]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [38]:
formula = 'deathRate ~ hospitalizedRate'
regressor = 'hospitalizedRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [39]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [40]:
dhr = estimates

<br>

Prevailing

In [41]:
dhr.loc[:, 'prevailingDHR'] = 100 * dhr['gradient']
dhr.loc[:, 'rank'] = dhr['prevailingDHR'].rank(method='min', ascending=False)
dhr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [42]:
dhr = dhr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'deathHospitalizedRate']], 
                how='left', on=['datetimeobject', 'STUSPS'])
dhr.loc[:, 'prevailingHPD'] = np.where(dhr['prevailingDHR'] > 0, dhr['prevailingDHR'].rdiv(100), 0)

dhr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'deathHospitalizedRate': 'continuousDHR'}, 
           inplace=True)

dhr.to_csv(path_or_buf=os.path.join(warehouse, 'dhr.csv'), index=False, header=True, encoding='utf-8')
dhr.to_json(path_or_buf=os.path.join(warehouse, 'dhr.json'), orient='values')

dhr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDHR,rank,continuousDHR,prevailingHPD
32,2021-02-05,0.239916,0.205315,0.263003,3.62514e-22,0.995677,-9.2592,MD,23.9916,33.0,22.293266,4.16813
33,2021-02-05,0.211647,0.115432,0.293235,7.20896e-14,0.972144,17.2156,CO,21.1647,34.0,25.626741,4.72485
34,2021-02-05,0.194472,0.131952,0.248922,5.82048e-15,0.975383,53.4102,RI,19.4472,35.0,26.121871,5.14212
35,2021-02-05,0.181233,0.140658,0.235627,6.54162e-16,0.98571,-24.1199,UT,18.1233,36.0,12.562704,5.51777
36,2021-02-05,0.17608,0.0886174,0.53676,3.58663e-10,0.895544,121.433,NJ,17.608,37.0,35.683890,5.67924


<br>
<br>

## Clean-up

In [43]:
!rm -rf *.log
!rm -rf *.pdf
!rm -rf states

<br>
<br>

## End

In [44]:
%%bash

date +"%Y-%m-%d %T"

2021-02-06 22:48:45
